In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math
import keras as kr
import random 
import model_prep_withFE
import model_prep_withoutFE
import operator
import lightgbm as lgb
import statsmodels.api as sm
from model_prep_withFE import prepare_data, get_features, get_bounds, get_interval_accuracy_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, GridSearchCV, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Ridge, LassoCV, Lasso
from sklearn import tree
from sklearn import metrics
from sklearn import preprocessing
import keras.callbacks as cb
from lightgbm import LGBMRegressor
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.optimizers import RMSprop
from keras.optimizers import Adam

# Load in pre-split data
X_train, X_test, y_train, y_test = prepare_data()
features = get_features()
train_bounds_5 = get_bounds(y_train, 5)
train_bounds_10 = get_bounds(y_train, 10)
test_bounds_5 = get_bounds(y_test, 5)
test_bounds_10 = get_bounds(y_test, 10)

Train data(range):
2019-04-11 21:50:23
2019-08-22 19:06:11
Test data(range):
2019-04-11 21:50:04
2019-08-22 19:05:50


In [3]:
X_train_df = pd.DataFrame(X_train)
X_train_df.columns = features
corr = X_train_df[features].corr()
corr = pd.melt(corr.reset_index(), id_vars='index')
corr.columns = ['x', 'y', 'value']

In [4]:
duplicate = []
for i in range(len(corr)):
    if corr.iloc[i]['x'] == corr.iloc[i]['y']:
        duplicate.append(i)

In [5]:
corr = corr.drop(duplicate, axis=0)
corr = corr.dropna()
corr = corr.reset_index(drop=True)

In [6]:
inds = list(corr.value.abs().argsort()[::-1])

In [7]:
corr.iloc[inds][0:50]

,x,y,value
2192,fare_Flexible,fare_Promo,-0.780959
2142,fare_Promo,fare_Flexible,-0.780959
101,originpop,destinationpop,-0.650508
51,destinationpop,originpop,-0.650508
205,month,days_from_holiday,0.646344
304,days_from_holiday,month,0.646344
1450,travel_mins,train_type_AVE,-0.626145
28,train_type_AVE,travel_mins,-0.626145
2249,train_class_Turista con enlace,fare_Promo +,0.625875
2544,fare_Promo +,train_class_Turista con enlace,0.625875


In [62]:
drop = ['train_class_Cama G. Clase', 'destination_MADRID', 'fare_Adulto Ida']

In [131]:
# read in data and drop unnamed, start/end date columns
renfedata = pd.read_csv("input/cleaned_data.csv")
# sample data to test newly reduced feature dataset
renfedata = renfedata.sample(n=50000)
renfedata = renfedata.drop(columns=['Unnamed: 0','start_date', 'end_date', 'insert_date', 'train_class_Cama G. Clase','fare_Individual Sleeper-Flexible', 'destination_MADRID', 'origin_MADRID', 'fare_Adulto Ida'], axis=1)
features = renfedata.drop(columns=['price'], axis=1)
response = renfedata[['price']]
X_train, X_test, y_train, y_test = train_test_split(features, response, train_size=0.75)
minMax_scaler = preprocessing.MinMaxScaler().fit(X_train)
X_train_scaled = minMax_scaler.transform(X_train)
X_test_scaled = minMax_scaler.transform(X_test)

In [134]:
model = GradientBoostingRegressor(n_estimators=500, max_depth=8, loss='ls', min_samples_split=2, learning_rate=0.01)

In [135]:
y_train = list(np.array(y_train).flatten())
y_test = list(np.array(y_test).flatten())

In [136]:
model.fit(X_train_scaled, y_train)
model.score(X_test_scaled, y_test)

0.9374538812734796

In [137]:
mean_squared_error(y_test, model.predict(X_test_scaled))

40.9509677289871

In [133]:
get_interval_accuracy_score(test_bounds_5, model.predict(X_test_scaled))